In [1]:
%pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 48.6 MB/s eta 0:00:00


In [2]:
#Script that solves a linear programming problem using PuLP
# for single machine problem with constrains

# Import PuLP modeler functions
from pulp import *

# Create the 'prob' variable to contain the problem data
prob = LpProblem("SM",LpMinimize)
# sets
# 19 jobs
J = ["J1","J2","J3","J4","J5","J6","J7","J8","J9","J10","J11","J12","J13","J14","J15","J16","J17","J18","J19"]
p=[15.0, 14.0, 15.0, 4.0, 13.0, 8.0, 16.0, 8.0, 12.0, 5.0, 14.0, 12.0, 16.0, 6.0, 4.0, 16.0, 8.0, 11.0, 3.0]
d=[8.0, 4.0, 7.0, 5.0, 15.0, 12.0, 3.0, 7.0, 15.0, 6.0, 9.0, 17.0, 14.0, 5.0, 20.0, 14.0, 8.0, 16.0, 16.0]
p = makeDict([J],p)
d = makeDict([J],d)
# Create problem variables
# S_j = start time of job j
# C_j = completion time of job j
# Seq = Sequence variable that
# T_j = Tardiness of job j
# E_j = Earliness of job j
S = LpVariable.dicts("S",(j for j in J),lowBound=0,cat='Integer')
C = LpVariable.dicts("C",(j for j in J),lowBound=0,cat='Integer')
T = LpVariable.dicts("T",(j for j in J),lowBound=0,cat='Integer')
E = LpVariable.dicts("E",(j for j in J),lowBound=0,cat='Integer')
Tmax = LpVariable("Tmax",lowBound=0,cat='Integer')
Emax = LpVariable("Emax",lowBound=0,cat='Integer')

# Completion time of each job
for j in J:
    # get the completion time following the sequence
    prob += C[j] == S[j] + p[j], "C[%s]" % j
    # get the tardiness
    prob += T[j] >= C[j] - d[j], "T[%s]" % j
    # get the earliness
    prob += E[j] >= d[j] - C[j], "E[%s]" % j
    # get the maximum tardiness
    prob += Tmax >= T[j] , "Tmax[%s]" % j
    # get the maximum earliness
    prob += Emax >= E[j] , "Emax[%s]" % j
    
# precedences
# j17 must end before 16 starts
prob += C["J17"] <= S["J16"]
# J17 must end before 6 starts
prob += C["J17"] <= S["J6"]
# J18 must end before 16 starts
prob += C["J18"] <= S["J16"] 
# J19 must end before 16 starts
prob += C["J19"] <= S["J16"]
# J19 must end before 8 starts
prob += C["J19"] <= S["J8"]
# J16 must end before 14 starts
prob += C["J16"] <= S["J14"]
# J16 must end before 15 starts
prob += C["J16"] <= S["J15"]
# J14 must end before 13 starts
prob += C["J14"] <= S["J13"]
# J15 must end before 13 starts
prob += C["J15"] <= S["J13"]
# J13 must end before 12 starts
prob += C["J13"] <= S["J12"]
# J13 must end before 11 starts
prob += C["J13"] <= S["J11"]
# J13 must end before 10 starts
prob += C["J13"] <= S["J10"]
# J12 must end before 9 starts
prob += C["J12"] <= S["J9"]
# J11 must end before 9 starts
prob += C["J11"] <= S["J9"]
# J10 must end before 9 starts
prob += C["J10"] <= S["J9"]
# J9 must end before 7 starts
prob += C["J9"] <= S["J7"]
# J7 must end before 2 starts
prob += C["J7"] <= S["J2"]
# J7 must end before 3 starts
prob += C["J7"] <= S["J3"]
# J6 must end before 1 starts
prob += C["J6"] <= S["J1"]
# J6 must end before 2 starts
prob += C["J6"] <= S["J2"]
# J8 must end before 2 starts
prob += C["J8"] <= S["J2"]
# J8 must end before 4 starts
prob += C["J8"] <= S["J4"]

#EDD Earliness Due Date
#Job 17 must start first
prob += S["J17"] == 0
#Job 18 must start after 17
prob += S["J18"] >= C["J17"]
#Job 19 must start after 18
prob += S["J19"] >= C["J18"]

#Job 15 must start after 14
prob += S["J15"] >= C["J14"]

#Job 10 must start after 13
prob += S["J10"] >= C["J13"]
#Job 11 must start after 10
prob += S["J11"] >= C["J10"]
#Job 12 must start after 11
prob += S["J12"] >= C["J11"]

#Job 8 must start after 7
prob += S["J8"] >= C["J7"]
#Job 6 must start after 8
prob += S["J6"] >= C["J8"]
#Job 2 must start after 6
prob += S["J2"] >= C["J6"]
#Job 4 must start after 2
prob += S["J4"] >= C["J2"]
#Job 3 must start after 4
prob += S["J3"] >= C["J4"]
#Job 1 must start after 3
prob += S["J1"] >= C["J3"]
#Job 5 must start after 1
prob += S["J5"] >= C["J1"]

# Objective function minimizes the max tardiness
prob += Tmax, "Objective function"
# The problem data is written to an .lp file
prob.writeLP("SM.lp")
# The problem is solved using PuLP's choice of Solver
prob.solve()
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)
# The optimised objective function value is printed to the screen
print("Tardiness = ", value(prob.objective))
#Create a dictionary to hold the sequence of jobs ordered by start time
Seq = {}
#Create a dictionary to hold the start time of each job
Start = {}
#Create a dictionary to hold the completion time of each job
Comp = {}
#Put the sequence of jobs in the dictionary
for v in prob.variables():
    if v.name[0] == "S":
        Start[v.name[2:]] = v.varValue
    if v.name[0] == "C":
        Comp[v.name[2:]] = v.varValue
#Sort the jobs by start time
for key, value in sorted(Start.items(), key=lambda item: item[1]):
    Seq[key] = value
#Print the sequence of jobs
print("Sequence of jobs")
for key, value in Seq.items():
    print(key, value)




Status: Optimal
C_J1 = 187.0
C_J10 = 69.0
C_J11 = 83.0
C_J12 = 95.0
C_J13 = 64.0
C_J14 = 44.0
C_J15 = 48.0
C_J16 = 38.0
C_J17 = 8.0
C_J18 = 19.0
C_J19 = 22.0
C_J2 = 153.0
C_J3 = 172.0
C_J4 = 157.0
C_J5 = 200.0
C_J6 = 139.0
C_J7 = 123.0
C_J8 = 131.0
C_J9 = 107.0
E_J1 = 0.0
E_J10 = 0.0
E_J11 = 0.0
E_J12 = 0.0
E_J13 = 0.0
E_J14 = 0.0
E_J15 = 0.0
E_J16 = 0.0
E_J17 = 0.0
E_J18 = 0.0
E_J19 = 0.0
E_J2 = 0.0
E_J3 = 0.0
E_J4 = 0.0
E_J5 = 0.0
E_J6 = 0.0
E_J7 = 0.0
E_J8 = 0.0
E_J9 = 0.0
Emax = 0.0
S_J1 = 172.0
S_J10 = 64.0
S_J11 = 69.0
S_J12 = 83.0
S_J13 = 48.0
S_J14 = 38.0
S_J15 = 44.0
S_J16 = 22.0
S_J17 = 0.0
S_J18 = 8.0
S_J19 = 19.0
S_J2 = 139.0
S_J3 = 157.0
S_J4 = 153.0
S_J5 = 187.0
S_J6 = 131.0
S_J7 = 107.0
S_J8 = 123.0
S_J9 = 95.0
T_J1 = 179.0
T_J10 = 63.0
T_J11 = 74.0
T_J12 = 78.0
T_J13 = 50.0
T_J14 = 39.0
T_J15 = 28.0
T_J16 = 24.0
T_J17 = 0.0
T_J18 = 3.0
T_J19 = 6.0
T_J2 = 149.0
T_J3 = 165.0
T_J4 = 152.0
T_J5 = 185.0
T_J6 = 127.0
T_J7 = 120.0
T_J8 = 124.0
T_J9 = 92.0
Tmax = 185.0
Tardines